In [ ]:
import importlib
import local_utils; importlib.reload(local_utils)
from local_utils import *

train_df = pd.read_csv(DATA_DIR + '/train_v2.csv')
test_df = pd.read_csv(DATA_DIR + '/sample_submission_v2.csv')

label_map, inv_label_map, Y = process_labels(train_df)
print("Shape of Y {}".format(Y.shape))

train_inx, valid_inx = stratified_sampling(Y)
print("Train {}; Valid {};".format(len(train_inx), len(valid_inx)))

w_size = 128
h_size = 128
input_shape = (w_size, h_size, 3)
print(input_shape)

In [ ]:
start = time()
X = load_images(train_df['image_name'].values, DATA_DIR + '/train-jpg', True, w_size, h_size)
print(X.shape)
print(time() - start)

In [5]:
model = Sequential()
model.add(BatchNormalization(input_shape=input_shape))
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(17, (3, 3), padding='same'))
# model.add(Dropout(0.25))
model.add(GlobalMaxPooling2D())
model.add(Activation('sigmoid'))

# model.add(Flatten())
# model.add(Dense(512, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))
# model.add(Dense(17, activation='sigmoid'))

In [6]:
batch_size = 64
model.compile(optimizer=optimizers.Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 128, 128, 3)       12        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
batch_normalization_2 (Batch (None, 128, 128, 32)      128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 126, 126, 32)      9248      
_________________________________________________________________
batch_normalization_3 (Batch (None, 126, 126, 32)      128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 63, 63, 32)        0         
__________

In [7]:
model.fit(X[train_inx], Y[train_inx],
          batch_size=batch_size,
          epochs=3,
          verbose=1,
          validation_data=(X[valid_inx], Y[valid_inx]))

Train on 36431 samples, validate on 4048 samples
Epoch 1/3
36431/36431 [==============================] - 220s - loss: 0.2092 - acc: 0.9192 - val_loss: 0.2026 - val_acc: 0.9302
Epoch 2/3
36431/36431 [==============================] - 218s - loss: 0.1579 - acc: 0.9377 - val_loss: 0.1621 - val_acc: 0.9390
Epoch 3/3
36431/36431 [==============================] - 218s - loss: 0.1439 - acc: 0.9436 - val_loss: 0.1424 - val_acc: 0.9470


In [8]:
model.optimizer.lr = 1e-4
model.fit(X[train_inx], Y[train_inx],
          batch_size=batch_size,
          epochs=3,
          verbose=1,
          validation_data=(X[valid_inx], Y[valid_inx]))

Train on 36431 samples, validate on 4048 samples
Epoch 1/3
36431/36431 [==============================] - 218s - loss: 0.1348 - acc: 0.9475 - val_loss: 0.1349 - val_acc: 0.9491
Epoch 2/3
36431/36431 [==============================] - 218s - loss: 0.1282 - acc: 0.9500 - val_loss: 0.1300 - val_acc: 0.9512
Epoch 3/3
36431/36431 [==============================] - 218s - loss: 0.1243 - acc: 0.9519 - val_loss: 0.1302 - val_acc: 0.9508


In [9]:
model.optimizer.lr = 1e-5
model.fit(X[train_inx], Y[train_inx],
          batch_size=batch_size,
          epochs=3,
          verbose=1,
          validation_data=(X[valid_inx], Y[valid_inx]))

Train on 36431 samples, validate on 4048 samples
Epoch 1/3
36431/36431 [==============================] - 218s - loss: 0.1199 - acc: 0.9536 - val_loss: 0.1261 - val_acc: 0.9525
Epoch 2/3
36431/36431 [==============================] - 218s - loss: 0.1174 - acc: 0.9546 - val_loss: 0.1280 - val_acc: 0.9508
Epoch 3/3
36431/36431 [==============================] - 218s - loss: 0.1144 - acc: 0.9558 - val_loss: 0.1183 - val_acc: 0.9555


In [10]:
model.save(DATA_DIR + '/models/simple_fcn_model.h5', include_optimizer=False)

In [14]:
save_array(DATA_DIR + '/models/simple_fcn_model_thres.dat', thres)